In [1]:
# -*- coding: utf-8 -*-
import ipywidgets as widgets
from ipywidgets import interact, SelectMultiple, fixed, Checkbox, IntRangeSlider, IntSlider, FloatSlider
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from src.DataVisualization import DataVisualization

data_visualization = DataVisualization()
data_visualization.season_visualization(2017)

interactive(children=(IntSlider(value=2, continuous_update=False, description='type', max=3, min=2), IntSlider…